In [1]:
import sys, os
from pathlib import Path
if '_old_cwd' not in locals():
    _old_cwd = os.getcwd()
    os.chdir(Path(os.getcwd()).parent)
    sys.path.append(os.getcwd())
# from ib_insync import *
# from tqdm.notebook import tqdm
from omnibelt import tqdmd_notebook as tqdm
from tabulate import tabulate
from omnibelt import save_json, load_json
from collections import Counter
import xmltodict

from src.euro import IB_Extractor


In [2]:
root = Path(os.getcwd()) / 'ibkr'

In [3]:
ibe = IB_Extractor(True)

ERROR:ib_insync.wrapper: Error 321, reqId 2147483647: Error validating request.-'cB' : cause - The API interface is currently in Read-Only mode.
ERROR:ib_insync.wrapper: Error 321, reqId -1: Error validating request.-'b_' : cause - The API interface is currently in Read-Only mode.
ERROR:ib_insync.ib: open orders request timed out
ERROR:ib_insync.ib: completed orders request timed out


In [4]:
raw = ['ASML', 'MC', 'TTE', 'SAP', 'SIE', 'SAN', 'OR', 'SU', 'ALV', 'AI', 'AIR', 'RMS', 'BNP', 'DTE', 'IBE', 'DG', 'SAN1', 'MBG', 'SAF', 'EL', 'CS', 'IFX', 'BAYN', 'PRX', 'ENEL', 'ADYEN', 'INGA', 'ABI', 'MUV2', 'DHL', 'UCG', 'STLAP', 'BBVA', 'BAS', 'ISP', 'RI', 'CRH', 'ITX', 'KER', 'NDA', 'BMW', 'BN', 'DB1', 'FLTR', 'ENI', 'ADS', 'AD', 'VOW3', 'NOKIA', 'VNA']
wts = [8.22, 6.88, 4.3, 4.22, 3.63, 3.47, 3.21, 2.89, 2.74, 2.67, 2.46, 2.22, 2.2, 2.15, 2.09, 1.97, 1.89, 1.82, 1.79, 1.73, 1.69, 1.63, 1.63, 1.57, 1.52, 1.51, 1.5, 1.46, 1.46, 1.44, 1.4, 1.37, 1.36, 1.36, 1.33, 1.27, 1.27, 1.21, 1.18, 1.16, 1.11, 1.11, 1.03, 1.0, 0.99, 0.96, 0.96, 0.78, 0.59, 0.45]

In [5]:
cts_data = {}

In [6]:
pbar = tqdm(raw)
for sym in pbar:
    if sym in cts_data:
        continue
    pbar.set_description(sym)
    cts_data[sym] = ibe.find_all_contracts(sym)

  0%|          | 0/50 [00:00<?, ?it/s]

In [16]:
def extract_contract_info(contract):
    return {'currency': contract.currency, 'exchange': contract.exchange, 'symbol': contract.symbol, 'secType': contract.secType, 'conId': contract.conId, 'primaryExchange': contract.primaryExchange}
def contract_from_info(info):
    return Contract(conId=info['conId'], symbol=info['symbol'], secType=info['secType'], currency=info['currency'], exchange=info['exchange'], primaryExchange=info['primaryExchange'])

In [8]:
cts_options = [[extract_contract_info(c) for c in cts_data[sym]] for sym in raw]
len(cts_options)

50

In [9]:
by_ID = {}
for k, vs in cts_data.items():
    for v in vs:
        if v.conId not in by_ID:
            by_ID[v.conId] = extract_contract_info(v)

In [10]:
# save_json(by_ID, root / 'all-stoxx-contracts.json');

In [11]:
by_ID = load_json(root / 'all-stoxx-contracts.json')
len(by_ID)

130

In [12]:
eur = [v for v in by_ID.values() if v['currency'] == 'EUR']
len(eur)

50

In [17]:
snaps = [ibe.snapshot(contract_from_info(e)) for e in tqdmd(eur, lambda v: v['symbol'])]

  0%|          | 0/50 [00:00<?, ?it/s]

In [19]:
snap_dicts = [xmltodict.parse(s) for e, s in zip(eur, snaps)]
len(snap_dicts)

50

In [22]:
# save_json(snap_dicts, root / 'all-stoxx-snapshots.json')

In [23]:
out = xmltodict.parse(ibe.recommendations(contract_from_info(eur[0])))

In [15]:
from pathlib import Path
import pandas as pd
from omnibelt import load_csv, save_json, load_json, load_yaml, save_yaml
from stox import misc

In [48]:
root = misc.assets_root() / 'dis'
base_path = root / 'stoxx50-oct9.yaml'
# path = root / 'stoxx50-oct9.json'

In [49]:
base = load_yaml(base_path)
tks = [v['ticker'] for v in base['tickers']]
len(tks), len(set(tks))

(241, 241)

In [11]:
# assert path.exists()
# base = load_yaml(base_path)
# raw = load_json(path)
# len(raw)

49

In [13]:
report = {'tickers': []}
for k, v in raw.items():
    report['tickers'].append({'ticker': k, 'sel': True, 'weight': v})
report['tickers'].extend([{'ticker': v['ticker']} for v in base['tickers'] if v['ticker'] not in raw])
len(report['tickers'])

241

In [14]:
# save_yaml(report, root / 'stoxx50-oct9.yaml')

In [16]:
stats = pd.read_csv(root / '..' / 'euro-stats-sep28.csv')
stats

,Ticker,Company Name,Country,Sector,Industry,10 Day Average Volume,Price,Current Shares,Dividend Yield,Recommendation Key,...,Forward P/E,Trailing EPS,Forward EPS,52 Week Change,"52 Week Relative Price (1=52 week high, 0=52 week low)",Log Market Cap,Overall Risk (1-10),Percent of Shares Held by Institutions,Percent of Shares Held by Insiders,Company Description
0,0EXG.IL,"Jerónimo Martins, SGPS, S.A.",Portugal,Consumer Defensive,Food Distribution,230674,21.6 EUR,22,2.55 %,none,...,NaN,1.09,NaN,+3.16 %,0.366931,10.159659,10.0,24.534000,56.183000,"Jerónimo Martins, SGPS, S.A. operates in the f..."
1,1COV.DE,Covestro AG,Germany,Basic Materials,Specialty Chemicals,1011680,50.7 EUR,0,0 %,buy,...,19.045113,-4.47,2.66,+73.3 %,0.870405,9.980529,2.0,55.899000,4.578000,Covestro AG supplies high-tech polymer materia...
2,ABI.BR,Anheuser-Busch InBev SA/NV,Belgium,Consumer Defensive,Beverages—Brewers,1314375,50.9 EUR,3,1.46 %,buy,...,15.230539,2.87,3.34,+8.39 %,0.323413,11.009414,8.0,14.675000,48.661998,"Anheuser-Busch InBev SA/NV produces, distribut..."
3,ABN.AS,ABN AMRO Bank N.V.,Netherlands,Financial Services,Banks—Diversified,3428291,13.2 EUR,0,9.84 %,hold,...,5.303213,2.71,2.49,+45.5 %,0.543758,10.073495,3.0,39.771000,6.647000,ABN AMRO Bank N.V. provides various banking pr...
4,AC.PA,Accor SA,France,Consumer Cyclical,Lodging,745791,32.4 EUR,0,2.2 %,buy,...,16.175000,2.03,2.00,+48.9 %,0.788904,9.951598,8.0,52.854997,20.865000,Accor SA operates a chain of hotels worldwide....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,WDP.BR,Warehouses De Pauw,Belgium,Real Estate,REIT—Industrial,312079,22.7 EUR,25,4.31 %,hold,...,15.791666,-0.45,1.44,-3.89 %,0.015152,9.684880,1.0,35.825002,21.603000,WDP develops and invests in logistics real est...
235,WIE.VI,Wienerberger AG,Austria,Basic Materials,Building Materials,196126,23.8 EUR,0,3.82 %,buy,...,7.745929,4.41,3.07,+15.7 %,0.388303,9.398999,2.0,55.573004,1.301000,"Wienerberger AG produces and sells bricks, roo..."
236,WKL.AS,Wolters Kluwer N.V.,Netherlands,Industrials,Specialty Business Services,533386,113 EUR,0,1.67 %,buy,...,23.604166,4.18,4.80,+16.7 %,0.631463,10.440227,1.0,66.273004,0.041000,Wolters Kluwer N.V. provides professional info...
237,WLN.PA,Worldline SA,France,Technology,Software—Infrastructure,748223,27 EUR,0,0 %,buy,...,9.418119,0.83,2.87,-35.6 %,0.062696,9.905877,2.0,55.799000,11.505999,Worldline SA provides payments and transaction...


In [23]:
df = stats[['Ticker', 'Price', 'Current Shares']]#.to_dict()
df

,Ticker,Price,Current Shares
0,0EXG.IL,21.6 EUR,22
1,1COV.DE,50.7 EUR,0
2,ABI.BR,50.9 EUR,3
3,ABN.AS,13.2 EUR,0
4,AC.PA,32.4 EUR,0
...,...,...,...
234,WDP.BR,22.7 EUR,25
235,WIE.VI,23.8 EUR,0
236,WKL.AS,113 EUR,0
237,WLN.PA,27 EUR,0


In [27]:
# parse prices
df['price'] = df.apply(lambda r: 1000.**('k' in r['Price'].lower()) * float(r['Price'].replace(' EUR', '').replace('k', '').strip()), axis=1)

C:\Users\anwan\AppData\Local\Temp\ipykernel_29744\2794366516.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'] = df.apply(lambda r: 1000.**('k' in r['Price'].lower()) * float(r['Price'].replace(' EUR', '').replace('k', '').strip()), axis=1)


In [29]:
df['values'] = df.apply(lambda r: r['price'] * r['Current Shares'], axis=1)
df

C:\Users\anwan\AppData\Local\Temp\ipykernel_29744\3214104937.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['values'] = df.apply(lambda r: r['price'] * r['Current Shares'], axis=1)


,Ticker,Price,Current Shares,price,values
0,0EXG.IL,21.6 EUR,22,21.6,475.2
1,1COV.DE,50.7 EUR,0,50.7,0.0
2,ABI.BR,50.9 EUR,3,50.9,152.7
3,ABN.AS,13.2 EUR,0,13.2,0.0
4,AC.PA,32.4 EUR,0,32.4,0.0
...,...,...,...,...,...
234,WDP.BR,22.7 EUR,25,22.7,567.5
235,WIE.VI,23.8 EUR,0,23.8,0.0
236,WKL.AS,113 EUR,0,113.0,0.0
237,WLN.PA,27 EUR,0,27.0,0.0


In [31]:
total = df['values'].sum()

In [33]:
df['pct'] = df.apply(lambda r: 100 * r['values'] / total, axis=1)
df

C:\Users\anwan\AppData\Local\Temp\ipykernel_29744\1728317906.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pct'] = df.apply(lambda r: 100 * r['values'] / total, axis=1)


,Ticker,Price,Current Shares,price,values,pct
0,0EXG.IL,21.6 EUR,22,21.6,475.2,1.312861
1,1COV.DE,50.7 EUR,0,50.7,0.0,0.000000
2,ABI.BR,50.9 EUR,3,50.9,152.7,0.421873
3,ABN.AS,13.2 EUR,0,13.2,0.0,0.000000
4,AC.PA,32.4 EUR,0,32.4,0.0,0.000000
...,...,...,...,...,...,...
234,WDP.BR,22.7 EUR,25,22.7,567.5,1.567864
235,WIE.VI,23.8 EUR,0,23.8,0.0,0.000000
236,WKL.AS,113 EUR,0,113.0,0.0,0.000000
237,WLN.PA,27 EUR,0,27.0,0.0,0.000000


In [44]:
tks = []
for i, row in df.iterrows():
    row = row.to_dict()
    tks.append({'ticker': row['Ticker'], 'weight': row['pct']})
# tks.extend([{'ticker': v['ticker']} for v in base['tickers'] if v['ticker'] not in df['Ticker']])
tks.sort(key=lambda v: (v.get('weight',0.), v['ticker']), reverse=True)
len(tks)

239

In [45]:
# save_yaml({'tickers': tks}, root / 'ibkr-full-jan23.yaml')